In [49]:

import csv
import numpy as np
import re
import pandas as pd
import glob
from itertools import chain
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import *
from datetime import datetime, timedelta
import py2neo as p2neo
import sys
from pymongo import MongoClient
sys.path.append("/PyLib/strictrfc3339")
import strict_rfc3339


NameError: name 'py2neo' is not defined

In [7]:
def getLog(fileName):
    """
    Ouvre le fichier et crèe une liste avec un log par ligne
    """
    results = []
    with open(fileName, newline='') as inputfile:
        for row in csv.reader(inputfile):
            results.append(row)
    return results

In [8]:
# Liste des fichiers logs
listfiles = glob.glob('/home/joseph/Dropbox/DeepLearning/ProjetFilRouge/AirFrance/logs/logs/*')

In [15]:
# Utilise le module Glob pour Lister tous les fichiers logs et les ouvrir
res = []
for filename in listfiles:
    res.append(getLog(filename))

# Flatten ma liste --> passer d'une "liste de liste" à une liste simple
logs = list(chain(*res))

# Split les log en fonction des 3 premiersd espaces rencontrés
logSplit = list(map(lambda x : logs[x][0].split(' ', 3),
                    range(len(logs))))

# Création d'un DataFrame à la place de la liste Splitée
df = pd.DataFrame(logSplit, columns=["Date", "Hostname", "Command", "message"])

# Cleaning
df["Date"] = df["Date"].apply(strict_rfc3339.rfc3339_to_timestamp)
df["Command"] = df["Command"].str[:-1]
df["ID"] = df["Command"].str.split("[").str[1].str[:-1]
df["Command"] = df["Command"].str.split("[").str[0]
df = df[["Date", "Hostname", "Command", "ID", "message"]]

# Index
df = df.set_index("Date")
df.head(20)

,Hostname,Command,ID,message
Date,,,,
1510272001,qvipcasb2c02,CROND,2475,(isashare) CMD (/exploit/ISA/share/scripts/bin...
1510272001,qvipcasb2c02,CROND,2474,(isashare) CMD (/exploit/ISA/share/scripts/bin...
1510272001,qvipcasb2c02,CROND,2476,(isashare) CMD (/exploit/ISA/share/scripts/bin...
1510272001,qvipcasb2c02,CROND,2477,(root) CMD (/usr/lib64/sa/sa1 1 1)
1510272001,qvipcasb2c02,CROND,2528,(a403023) CMD (/exploit/home/sar/qualif/Monito...
1510272001,qvipcasb2c02,crontab,2547,(a403023) LIST (a403023)
1510272001,qvipcasb2c02,CROND,2478,(isashare) CMD (/exploit/ISA/share/scripts/bin...
1510272002,qvipcasb2c02,sudo,NaN,isashare : TTY=unknown ; PWD=/tech/ICF/scripts...
1510272001,qvipcasb2c02,crontab,2546,(a403023) REPLACE (a403023)


# Extract IPs

In [31]:
import re

# Extract IP from string

def get_IP(string):
    try: 
        IP = re.search(r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}', string).group()
    except:
        IP='NaN'
    return IP

# Extract IP
df["IP"] = df["message"].apply(get_IP)

df_links['count'] = df[df['IP']!='NaN'].groupby(['Hostname','IP']).count()['ID']


In [32]:
df_links.drop(['Command', 'ID', 'message'],axis=1,inplace=True)
df_links.head()

count
Hostname     IP                 
qvipcasb2c01 10.60.10.46      64
             10.60.10.58    1140
             10.60.10.59    2088
             10.60.10.84      54
             10.60.105.51   4872

# Plot Neo4j Graph
Envoie les données du datframe précédent dans Neo4j pour la création de graphs du type (h:Hostname)-[CALLED_BY{count}]->(i:IP)

In [35]:
df = df[df['IP']!='NaN']
hostid_set = set(df['Hostname'].values)
ipid_set = set(df['IP'].values)

In [52]:
# ouvrir Neo4j Desktop et lancer une database sur le port 7474

g = Graph('http://localhost:7474', user='neo4j', password='AssouwillieIA2018@!+')

graph.delete_all()

tx = graph.begin()

for hostid in hostid_set:
    h = Node("Hostname", id=hostid)
    tx.create(h)

for ipid in ipid_set:
    i = Node("IP", id=ipid)
    tx.create(i)

for hostid, df_select in df_links.groupby(level=0):
    for ipid, count in df_select.groupby(level=1):
        count = int(np.mean(count.values))
        tx.run("MATCH (h:Hostname {id:\"" + hostid + "\"}) MATCH (i:IP {id:\"" +
               ipid + "\"}) CREATE (h)-[:CALLED_BY{count:\"" + str(count) + "\"}]->(i)")

tx.commit()

Unauthorized: http://localhost:7474/